In [13]:
from pyspark.sql import SparkSession

In [14]:
spark = ( 
 SparkSession
 .builder
    .master("local[*]")
 .appName('spark_sql')
 .getOrCreate()
)

In [15]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-11.csv')
)

In [16]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [17]:
df.createOrReplaceTempView('combustiveis')

In [18]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` IS NOT NULL
""").show()


+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [19]:
view_precos = spark.sql("""
    select 
        `Estado - Sigla`, 
        `Produto`, 
        regexp_replace(`Valor de Venda`, ",", ".") as `Valor de Venda`,
        `Unidade de Medida`
    from combustiveis
""")

In [20]:
view_precos.createOrReplaceTempView('view_precos')
view_precos.show(5)

+--------------+------------------+--------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Unidade de Medida|
+--------------+------------------+--------------+-----------------+
|            AL|          GASOLINA|           6.3|       R$ / litro|
|            AL|GASOLINA ADITIVADA|           6.3|       R$ / litro|
|            AL|            ETANOL|          4.88|       R$ / litro|
|            AL|          GASOLINA|          6.19|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6.29|       R$ / litro|
+--------------+------------------+--------------+-----------------+
only showing top 5 rows



In [21]:
df_diferenca_precos = spark.sql("""
    select 
        `Estado - Sigla`,
        `Produto`,
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor,
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca
   from view_precos
   group by all
   order by diferenca DESC
""")

In [22]:
df_diferenca_precos.show(5)

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       4.69|       8.69| 3.999999999999999|
|            SP|          GASOLINA|       R$ / litro|       4.49|       7.59|3.0999999999999996|
|            PA|            ETANOL|       R$ / litro|       3.85|        6.6|2.7499999999999996|
|            SP|            ETANOL|       R$ / litro|       2.78|       5.34|              2.56|
|            RS|            ETANOL|       R$ / litro|       3.88|       6.37|              2.49|
+--------------+------------------+-----------------+-----------+-----------+------------------+
only showing top 5 rows

